# Generate shape coefficients

The following steps are performed to obtain all the artifacts for force coefficient use case:

1. Read parameters file
2. Read Mesh description
   * Its path can be specified
   * Or it can be obtained from the simulation configuration file
3. Read data for pressure coefficient

Read parameters file

In [1]:
from cfdmod.use_cases.pressure.force.Cf_config import CfConfig
import pathlib

config_path=pathlib.Path("./fixtures/tests/pressure/Cf_params.yaml")
post_proc_cfg = CfConfig.from_file(config_path)

post_proc_cfg

{'measurement_1': CfConfig(bodies={'building': Body(surfaces=[], sub_bodies=SubBodyIntervals(x_intervals=[-inf, inf], y_intervals=[-inf, inf], z_intervals=[0.0, 10.0, 20.0]))}, variables=['Cfx', 'Cfy', 'Cfz'], statistics=['min', 'max', 'std', 'avg', 'skewness', 'kurtosis']),
 'measurement_2': CfConfig(bodies={'lanternim': Body(surfaces=['L1_xp', 'L2_yp', 'L3_zp_yp', 'L4_zp_ym', 'L5_ym', 'L6_xm'], sub_bodies=SubBodyIntervals(x_intervals=[-inf, inf], y_intervals=[-inf, inf], z_intervals=[-inf, inf]))}, variables=['Cfx', 'Cfy', 'Cfz'], statistics=['min', 'max', 'std', 'avg', 'skewness', 'kurtosis'])}

Read Mesh from file

In [2]:
from nassu.lnas import LagrangianFormat

mesh_path=pathlib.Path("./fixtures/tests/pressure/galpao/galpao.normalized.lnas")
mesh = LagrangianFormat.from_file(mesh_path)

mesh.name, mesh.geometry.vertices.shape, mesh.geometry.triangles.shape

('galpao', (1549, 3), (2915, 3))

Read pressure coefficient data

In [3]:
import pandas as pd

cp_data_path=pathlib.Path("./fixtures/tests/pressure/data/cp_t.resampled.hdf")
cp_data = pd.read_hdf(cp_data_path)

cp_data_to_use = (
    cp_data.to_frame() if isinstance(cp_data, pd.Series) else cp_data
)
cp_data_to_use

,time_step,point_idx,cp
0,10000.0,0,0.268273
1,10000.0,1,0.273546
2,10000.0,2,0.265319
3,10000.0,3,0.276913
4,10000.0,4,0.272593
...,...,...,...
294410,10100.0,2910,0.485344
294411,10100.0,2911,0.474767
294412,10100.0,2912,0.450817
294413,10100.0,2913,0.537118


Add areas to pressure coefficient data

In [4]:
areas = mesh.geometry._cross_prod() / 2
areas_df = pd.DataFrame({"Ax": areas[:, 0], "Ay": areas[:, 1], "Az": areas[:, 2]})
areas_df["point_idx"] = areas_df.index

cp_data = pd.merge(cp_data, areas_df,
    on="point_idx",
    how="left"
)

In [31]:
import numpy as np
from nassu.lnas import LagrangianGeometry
from cfdmod.use_cases.pressure.force.Cf_data import get_sub_body_index_mask

for cfg_label, cfg in post_proc_cfg.items():
    for body_label, body_cfg in cfg.bodies.items():
        if len(body_cfg.surfaces) == 0:
            # Include all surfaces
            geometry_idx = np.arange(0, len(mesh.geometry.triangles))
        else:
            # Filter mesh for all surfaces
            geometry_idx = np.array([], dtype=np.int32)
            for sfc in body_cfg.surfaces:
                if sfc not in mesh.surfaces.keys():
                    raise Exception("Surface defined in body is not separated in the LNAS file.")
                geometry_idx = np.concatenate((geometry_idx, mesh.surfaces[sfc])) 
                
        # verts_idx = np.unique(mesh.geometry.triangles[geometry_idx])
        body_triangle_verts = mesh.geometry._triangles_vertices[geometry_idx]
        full_verts = body_triangle_verts.reshape(len(body_triangle_verts) * 3, 3)
        verts, triangles = np.unique(full_verts, axis=0, return_inverse=True)
        
        print(body_triangle_verts.shape, full_verts.shape, verts.shape, triangles.shape)
        
        body_geom = LagrangianGeometry(
            vertices=verts,
            triangles=triangles
        )
        
        # body_geom = LagrangianGeometry(
        #     vertices=mesh.geometry.vertices.copy(),
        #     triangles=mesh.geometry.triangles[geometry_idx].copy()
        # )
        
        df_regions = body_cfg.sub_bodies.get_regions_df()
        
        sub_body_idx_array = get_sub_body_index_mask(body_geom, df_regions)
        sub_body_idx = pd.DataFrame({"point_idx": geometry_idx, "sub_body_idx": areas[:, 2]})
        
        body_data = cp_data[cp_data["point_idx"].isin(geometry_idx)].copy()
        body_data = pd.merge(body_data, sub_body_idx,
            on="point_idx",
            how="left"
        )
        
        body_data["fx"] = body_data["cp"] * body_data["Ax"]
        body_data["fy"] = body_data["cp"] * body_data["Ay"]
        body_data["fz"] = body_data["cp"] * body_data["Az"]
        
        # print(body_data)

(2915, 3, 3) (8745, 3) (1549, 3) (8745,)


ValueError: cannot reshape array of size 26235 into shape (8745,3,3)